In [1]:
import pandas as pd
import numpy as np
import pypsa

In [2]:
import glob

In [3]:
import os

In [4]:
iteration = 0

In [5]:
save_path = "resources/capacity_tables/0.csv"

In [8]:
climate_year = 1982

network_list = glob.glob("resources/networks/0/cy" + str(climate_year)+ "*.nc")

In [9]:
sorted(network_list)

['resources/networks/0/cy1982_ty2025.nc',
 'resources/networks/0/cy1982_ty2026.nc',
 'resources/networks/0/cy1982_ty2027.nc']

In [10]:
networks = pd.Series(
    index = [i.split("ty")[1][:4] for i in network_list]
)

networks.index = networks.index.astype(int)

In [11]:
for i, path in enumerate(network_list):
    networks.iloc[i] = pypsa.Network(path)

INFO:pypsa.io:Imported network cy1982_ty2025.nc has buses, generators, links, loads, storage_units
INFO:pypsa.io:Imported network cy1982_ty2027.nc has buses, generators, links, loads, storage_units
INFO:pypsa.io:Imported network cy1982_ty2026.nc has buses, generators, links, loads, storage_units


In [12]:
networks.sort_index(inplace=True)

In [13]:
capacities = pd.DataFrame()

In [14]:
for year in networks.index:
    capacities = pd.concat(
        [
            capacities,
            (
                networks[year].generators
                .set_index(
                    pd.Series(year, networks[year].generators.index),
                    append=True
                )
            )
        ]
    )

In [17]:
capacities.filter(like="new",axis=0)

,,bus,p_nom,carrier,entry,exit,efficiency,start_up_cost,ramp_limit_up,ramp_limit_down,p_min_pu,...,capital_cost,committable,shut_down_cost,stand_by_cost,up_time_before,down_time_before,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,
AL00 OCGT new 2025,2025,AL00,0.01,OCGT,2025.0,inf,0.42,20.36,7.2,7.2,0.4,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
AL00 CCGT new 2025,2025,AL00,0.01,CCGT,2025.0,inf,0.60,25.06,2.4,3.0,0.4,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
AT00 OCGT new 2025,2025,AT00,0.01,OCGT,2025.0,inf,0.42,20.36,7.2,7.2,0.4,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
AT00 CCGT new 2025,2025,AT00,0.01,CCGT,2025.0,inf,0.60,25.06,2.4,3.0,0.4,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
BA00 OCGT new 2025,2025,BA00,0.01,OCGT,2025.0,inf,0.42,20.36,7.2,7.2,0.4,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SK00 CCGT new 2025,2027,SK00,0.01,CCGT,2025.0,inf,0.60,25.06,2.4,3.0,0.4,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
UK00 OCGT new 2025,2027,UK00,0.01,OCGT,2025.0,inf,0.42,20.36,7.2,7.2,0.4,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0
UK00 CCGT new 2025,2027,UK00,0.01,CCGT,2025.0,inf,0.60,25.06,2.4,3.0,0.4,...,0.0,False,0.0,0.0,1,0,1.0,1.0,1.0,0.0


In [13]:
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [14]:
capacities.to_csv(save_path)